In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
train = pd.read_csv("data/train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train['FamilyMembers'] = train['SibSp'] + train['Parch']

In [4]:
features = ['Sex', 'FamilyMembers', 'Pclass', 'Embarked', 'Age', 'Cabin']
X = train[features]
y = train['Survived']
X.head()

,Sex,FamilyMembers,Pclass,Embarked,Age,Cabin
0,male,1,3,S,22.0,NaN
1,female,1,1,C,38.0,C85
2,female,0,3,S,26.0,NaN
3,female,1,1,S,35.0,C123
4,male,0,3,S,35.0,NaN


In [5]:
cat_cols = ['Sex', 'Embarked']
label_cols = ['Cabin']
num_cols = ['Age', 'FamilyMembers']

In [6]:
label_categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('labeler', OneHotEncoder(handle_unknown='ignore'))
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('label', label_categorical_transformer, label_cols),
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

In [8]:
model = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestClassifier(random_state=0, n_estimators=500, max_depth=5))])

In [9]:
model.fit(X,y)
y_pred = model.predict(X)

In [10]:
cross_val = cross_val_score(model, X, y)
cross_val, cross_val.mean()

(array([0.82122905, 0.80898876, 0.79775281, 0.79213483, 0.80337079]),
 0.804695248258113)

Most of the _Cabin_ values are _null_, so let's try again removing it from the features.

In [11]:
features = ['Sex', 'FamilyMembers', 'Pclass', 'Embarked', 'Age']
X = train[features]
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])
model = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestClassifier(random_state=0, n_estimators=500, max_depth=5))])

In [12]:
model.fit(X,y)
y_pred = model.predict(X)

In [13]:
cross_val = cross_val_score(model, X, y)
cross_val, cross_val.mean()

(array([0.82681564, 0.81460674, 0.80898876, 0.79775281, 0.84831461]),
 0.819295712761283)

In [16]:
test = pd.read_csv("data/test.csv")
test['FamilyMembers'] = test['SibSp'] + test['Parch']
test_data = test[features]
predictions = model.predict(test_data)
csv_output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
csv_output.to_csv('kaggle_submission.csv', index=False)